# Postgresql과 파이썬 연동(로컬에서 사용)

In [ ]:
# 로컬에서 설치

import psycopg2

In [ ]:
from typing import List
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
con = psycopg2.connect(host='localhost', dbname='postgres',user='postgres',
                      password='',port=5432)

In [ ]:
cur = con.cursor()

## 1. Create table

In [ ]:
# execute 실행시 오류가 나는 경우가 잦아 실행시키는 것이 좋음

cur.execute("ROLLBACK")
con.commit()

In [ ]:
# Title, year 정보를 담을 테이블 생성 쿼리

cur.execute('''
DROP TABLE IF EXISTS practice;
CREATE TABLE practice(
    title varchar(100),
    year Int4)
''')
con.commit()

In [ ]:
# execute 실행시 오류가 나는 경우가 잦아 실행시키는 것이 좋음

cur.execute("ROLLBACK")
con.commit()

In [ ]:
# Title, score 정보를 담을 테이블 생성 쿼리

cur.execute('''
DROP TABLE IF EXISTS embed_score;
CREATE TABLE embed_score(
    Title varchar(100),
    Score Float8)
''')
con.commit()

## 2. Load data to table

In [ ]:
# 실습에서 postgresql 사용할 때 csv로 다루었었고 neo4j 아웃풋을 csv로 내보낼 수 있어 예시로 csv 파일을 사용했습니다.

df = pd.read_csv('./prac1.csv', encoding='UTF-8')
df.columns = [c.lower() for c in df.columns] # 대문자 오류 발생할 수 있어 칼럼 소문자로 변경

In [ ]:
# 데이터를 postgresql로

df.to_sql(name = 'practice',
          con = con,
          schema = 'public',
          if_exists = 'append',
          index = False)

##----여기까지는 neo4j에서 파이썬을 거쳐 postgres로 가야할 경우에만 사용----##

## 3. Import data from postgresql

In [ ]:
df = pd.read_sql_table(table_name=practice, con=con)

## 4. Calculate score using embedding and load result to postgresql

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4", tags=None, options=None)

In [ ]:
title_lst = list(df['title'])

In [ ]:
embeddings = embed(title_lst)

In [ ]:
def get_recommendations(title):
  title_num = title_lst.index(title)
  tit_lst, score_lst = [], []
  for i in range(len(title_lst)):
    tit_lst.append(title_lst[i])
    score_lst.append(np.inner(embeddings[title_num], embeddings[i]))
  del tit_lst[title_num]
  del score_lst[title_num]
  dft = pd.DataFrame(tit_lst, columns = ['Title'])
  dfs = pd.DataFrame(score_lst, columns = ['Score'])
  return_df = pd.concat([dft,dfs],axis=1)
  return_df.to_sql(name = 'embed_score',
          con = con,
          schema = 'public',
          if_exists = 'append',
          index = False)

## 5. Recommendation result

In [ ]:
# 일반적인 쿼리

query = """
SELECT title, score
FROM embed_score
ORDER BY score DESC
LIMIT 20;
"""
result = execute(query)

In [ ]:
# 판다스로 다룰 때는 이것도 가능

pd.read_sql("select title score from embed_score order by score desc limit 20", product)